In [252]:
from composable import pipeable
from toolz.curried.operator import add
from toolz.curried.operator import eq
from composable.strict import map
from toolz.curried import get
import composable.strict as s
import re

In [253]:
with open('sell_short_trades.txt') as f:
    lines = f.readlines()
lines[:10]
no_header = lines[7:]


In [254]:
stripped_lines = [line.strip() for line in no_header if line.strip()]
stripped_lines[:20]

['CERS     SS      NEW        2.756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS',
 '/17  /17  256945       00',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO',
 'CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX',
 '/17  /17  866087       04',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 '854300',
 'CERS     SS      NEW          200   2.350000    2.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX',
 '/17  /17  866096       04',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 '854300',
 'CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 10:20:ContraSSFREX',
 '/17  /17  869780       23',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 '854300',
 'CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 10:22:ContraSSFREX',

In [255]:
header = re.compile('Symbol|TradeCommType')
page_num = re.compile('\d of \d\d')
extra_num = re.compile('854300')
is_page_num_line = lambda l: page_num.search(l)
is_blank = lambda l: len(l.strip()) == 0
is_extra = lambda l: extra_num.search(l)
is_header = lambda l: header.search(l)
needs_removal = lambda l: is_blank(l) or is_page_num_line(l) or is_extra(l) or is_header(l)
culled_lines = [l for l in stripped_lines if not needs_removal(l)]
culled_lines[:10]

['CERS     SS      NEW        2.756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS',
 '/17  /17  256945       00',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO',
 'CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX',
 '/17  /17  866087       04',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 'CERS     SS      NEW          200   2.350000    2.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX',
 '/17  /17  866096       04',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 'CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 10:20:ContraSSFREX']

In [256]:
get_layer = lambda l: l
joined = ','.join(get_layer(stripped_lines))
splitted = [line for line in joined]
joined
#splitted

'CERS     SS      NEW        2.756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS,/17  /17  256945       00,0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO,CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX,/17  /17  866087       04,0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS,854300,CERS     SS      NEW          200   2.350000    2.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX,/17  /17  866096       04,0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS,854300,CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 10:20:ContraSSFREX,/17  /17  869780       23,0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS,854300,CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 10:22:ContraSSFREX,/17  /17  892888       29,0704 PerUnit   TERM  0.010000     AA

In [257]:
stop_condition = lambda s: len(s) <= 3
get_layer = lambda l: l[:3]
comma_join = lambda l: ','.join(get_layer(l))
get_remaining = lambda l: l[3:]
new_seq = []
remaining_layers = culled_lines
while not stop_condition(remaining_layers):   
    new_seq = new_seq + [comma_join(get_layer(remaining_layers))]
    remaining_layers = get_remaining(remaining_layers)
new_seq[:10]

['CERS     SS      NEW        2.756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS,/17  /17  256945       00,0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO',
 'CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX,/17  /17  866087       04,0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 'CERS     SS      NEW          200   2.350000    2.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX,/17  /17  866096       04,0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 'CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 10:20:ContraSSFREX,/17  /17  869780       23,0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 'CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 10:22:ContraSSFREX,/17  /17  892888       29,0704 PerUnit   TERM  0.010000     AABBC 

In [258]:
stripped = [line.strip().split() for line in new_seq]
stripped[:5]
joined = '\n'.join([','.join(line) for line in stripped])
joined[:500]


'CERS,SS,NEW,2.756,2.400000,2.340000,2.45000006/1406/191706149900003,09:30:CustSS,/17,/17,256945,00,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=GSCO\nCERS,SS,NEW,100,2.36000018422.360000,2.37000006/1406/191706149900003,10:20:ContraSSFREX,/17,/17,866087,04,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS\nCERS,SS,NEW,200,2.350000,2.360000,2.37000006/1406/191706149900003,10:20:ContraSSFREX,/17,/17,866096,04,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS\nCERS,SS,NEW,100,2.350000'

In [259]:
with open('sell_short_trades_Grouped.csv','w') as outfile:
    outfile.write(joined)

!cat sell_short_trades_Grouped.csv | head -n 5

CERS,SS,NEW,2.756,2.400000,2.340000,2.45000006/1406/191706149900003,09:30:CustSS,/17,/17,256945,00,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=GSCO
CERS,SS,NEW,100,2.36000018422.360000,2.37000006/1406/191706149900003,10:20:ContraSSFREX,/17,/17,866087,04,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS
CERS,SS,NEW,200,2.350000,2.360000,2.37000006/1406/191706149900003,10:20:ContraSSFREX,/17,/17,866096,04,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS
CERS,SS,NEW,100,2.350000,2.350000,2.36000006/1406/191706149900003,10:20:ContraSSFREX,/17,/17,869780,23,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS
CERS,SS,NEW,100,2.350000,2.350000,2.36000006/1406/191706149900003,10:22:ContraSSFREX,/17,/17,892888,29,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS
cat: write error: Broken pipe


In [260]:
with open('sell_short_trades_Grouped.csv') as f:
    content = f.read()
g_lines = content.split('\n')
g_lines[:10]

['CERS,SS,NEW,2.756,2.400000,2.340000,2.45000006/1406/191706149900003,09:30:CustSS,/17,/17,256945,00,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=GSCO',
 'CERS,SS,NEW,100,2.36000018422.360000,2.37000006/1406/191706149900003,10:20:ContraSSFREX,/17,/17,866087,04,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS',
 'CERS,SS,NEW,200,2.350000,2.360000,2.37000006/1406/191706149900003,10:20:ContraSSFREX,/17,/17,866096,04,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS',
 'CERS,SS,NEW,100,2.350000,2.350000,2.36000006/1406/191706149900003,10:20:ContraSSFREX,/17,/17,869780,23,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS',
 'CERS,SS,NEW,100,2.350000,2.350000,2.36000006/1406/191706149900003,10:22:ContraSSFREX,/17,/17,892888,29,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS',
 'CERS,SS,NEW,100,2.360000,2.350000,2.36000006/1406/191706149900003,10:24:ContraSSFREX,/17,/17,909119,01,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS',
 'CERS,SS,NEW,100,2.360

In [261]:
expression = re.compile('(CERS|S|/17),.*')
[(i, l) for i, l in enumerate(g_lines) if not expression.search(l)]
                        

[]

In [262]:
expression = re.compile('.*,(SS|/17),.*')
[(i, l) for i, l in enumerate(g_lines) if not expression.search(l)]

[]

In [263]:
expression = re.compile('.*,(SS|/17),.*')
[(i, l) for i, l in enumerate(g_lines) if not expression.search(l)]

[]

In [264]:
expression1 = re.compile('(CERS|S|/17),(SS|/17),(NEW|\d\d\d\d\d\d),(\d\,\d\d\d|\d00),(\d\.\d*\,\d\.\d*|\d\.\d*\d\.\d*),(\d\.\d*/1406/\d*),(\d\d:\d\d:.*),(/17,/17|0704),(\d\d\d\d\d\d),(\d\d),(0704),(PerUnit),(TERM),(\d\.\d*),(AABBC),(\d*#STA=.*)')
expression2 = re.compile('(/17),(/17),(\d\d\d\d\d\d),(\d\d),(0704),(PerUnit),(TERM),(\d\.\d*),(AABBC),(\d*#STA=.*)')
expression3 = re.compile('(CERS|S),(SS),(NEW),(\d\,\d\d\d|\d00),(\d\.\d*\,\d\.\d*|\d\.\d*\d\.\d*),(\d\.\d*/1406/\d*),(\d\d:\d\d:.*),(0704),(PerUnit),(TERM),(\d\.\d*),(AABBC),(\d*#STA=.*),(\d\d\d\d\d\d),(\d\d),(0704),(PerUnit),(TERM),(\d\.\d*),(AABBC),(\d*#STA=),(/17,/17|0704),(\d\d\d\d\d\d),(\d\d),(CERS|S),(SS),(NEW),(\d\,\d\d\d|\d00),(\d\.\d*\,\d\.\d*|\d\.\d*\d\.\d*),(\d\.\d*/1406/\d*),(\d\d:\d\d:.*)')

in_file = lambda l: expression1.search(l) or expression2.search(l) or expression3.search(l)
[l for l in g_lines if not in_file]
                        

[]

In [265]:
all_start = [line.split(',') for line in g_lines]
all_start_joined = [','.join(line) for line in all_start]
#back_slash_removal = [line.split('/') for line in all_start_joined]
#back_slash_joined = [','.join(line) for line in back_slash_removal]
#colon_removal = [line.split(':') for line in all_start_joined]
#colon_joined = [','.join(line) for line in colon_removal]
pound_removal = [line.split('#') for line in all_start_joined]
pound_joined = [','.join(line) for line in pound_removal]
all_joined = pound_joined

In [293]:
Symbols = [line.split(',')[0] for line in all_joined if line[0] == 'C' or line[0] == 'S']
Sides = [line.split(',')[1] for line in all_joined if line[0] == 'C' or line[0] == 'S']
Cxls = [line.split(',')[2] for line in all_joined if line[0] == 'C' or line[0] == 'S']
Qtys = [line.split(',')[3] for line in all_joined if line[0] == 'C' or line[0] == 'S']
Prices = [line.split(',')[4] for line in all_joined if line[0] == 'C' or line[0] == 'S'] 
Bids = [line.split(',')[5] for line in all_joined if line[0] == 'C' or line[0] == 'S']  
Asks = [line.split(',')[5] for line in all_joined if line[0] == 'C' or line[0] == 'S']  
T_DatS_DatTradeIDs =[line.split(',')[6] for line in all_joined if line[0] == 'C' or line[0] == 'S'] 
TradeTiSSs =[line.split(',')[7] for line in all_joined if line[0] == 'C' or line[0] == 'S']
Exbkrs = [line.split(',')[8] for line in all_joined if line[0] == 'C' or line[0] == 'S']
TradeCommTypes = [line.split(',')[9] for line in all_joined if line[0] == 'C' or line[0] == 'S']
SourceCommissions = [line.split(',')[11] for line in all_joined if line[0] == 'C' or line[0] == 'S']
Accounts = [line.split(',')[12] for line in all_joined if line[0] == 'C' or line[0] == 'S']
OrderIDs = [line.split(',')[13] for line in all_joined if line[0] == 'C' or line[0] == 'S']     
GTLs = [line.split(',')[15] for line in all_joined if line[0] == 'C' or line[0] == 'S']
Trailer_Infos = [line.split(',')[16] for line in all_joined if line[0] == 'C' or line[0] == 'S']
Clrs = [line.split(',')[17] for line in all_joined if line[0] == 'C' or line[0] == 'S']

In [302]:
Symbols2 = [line.split(',') for line in all_joined if line[0] == '0']
Symbols2

[['0704',
  'PerUnit',
  'TERM',
  '0.010000',
  'AABBC',
  '1706149900003',
  'STA=CERS',
  '/17',
  '/17',
  '909224',
  '02',
  'CERS',
  'SS',
  'NEW',
  '100',
  '2.365000',
  '2.360000',
  '2.37000006/1406/191706149900003',
  '10:24:ContraSSFREX',
  '/17',
  '/17',
  '909672',
  '04'],
 ['0704',
  'PerUnit',
  'TERM',
  '0.010000',
  'AABBC',
  '1706149900003',
  'STA=CERS',
  'CERS',
  'SS',
  'NEW',
  '100',
  '2.360000',
  '2.360000',
  '2.37000006/1406/191706149900003',
  '10:24:ContraSSFREX',
  '/17',
  '/17',
  '910892',
  '11'],
 ['0704',
  'PerUnit',
  'TERM',
  '0.010000',
  'AABBC',
  '1706149900003',
  'STA=CERS',
  'CERS',
  'SS',
  'NEW',
  '100',
  '2.365000',
  '2.360000',
  '2.37000006/1406/191706149900003',
  '10:25:ContraSSFREX',
  '/17',
  '/17',
  '930540',
  '59'],
 ['0704',
  'PerUnit',
  'TERM',
  '0.010000',
  'AABBC',
  '1706149900003',
  'STA=CERS',
  'CERS',
  'SS',
  'NEW',
  '100',
  '2.365000',
  '2.360000',
  '2.37000006/1406/191706149900003',
  '10

In [308]:
Accounts2 = [line.split(',')[0] for line in all_joined if line[0] == '0']
OrderIDs2 = [line.split(',')[1] for line in all_joined if line[0] == '0']     
GTLs2 = [line.split(',')[2] for line in all_joined if line[0] == '0']
Trailer_Infos2 = [line.split(',')[3] for line in all_joined if line[0] == '0']
Clrs2 = [line.split(',')[4] for line in all_joined if line[0] == '0']
Symbols2 = [line.split(',')[7] for line in all_joined if line[0] == '0']
Sides2 = [line.split(',')[8] for line in all_joined if line[0] == '0']
Cxls2 = [line.split(',')[9] for line in all_joined if line[0] == '0']
Qtys2 = [line.split(',')[10] for line in all_joined if line[0] == '0']
Prices2 = [line.split(',')[11] for line in all_joined if line[0] == '0'] 
Bids2 = [line.split(',')[12] for line in all_joined if line[0] == '0']  
Asks2 = [line.split(',')[12] for line in all_joined if line[0] == '0']  
T_DatS_DatTradeIDs2 =[line.split(',')[13] for line in all_joined if line[0] == '0'] 
TradeTiSSs2 =[line.split(',')[14] for line in all_joined if line[0] == '0']
Exbkrs2 = [line.split(',')[15] for line in all_joined if line[0] == '0']
TradeCommTypes2 = [line.split(',')[5] for line in all_joined if line[0] == '0']
SourceCommissions2 = [line.split(',')[6] for line in all_joined if line[0] == '0']


In [309]:
csv_info2 = [(Symbol, Side, Cxl, Qty, Price, Bid, Ask, T_DatS_DatTradeID, TradeTiSS, Exbkr, 
                  TradeCommType, SourceCommission, Account, OrderID, GTL, Trailer_Info, Clr) 
        for Symbol, Side, Cxl, Qty, Price, Bid, Ask, T_DatS_DatTradeID, TradeTiSS, Exbkr, 
                  TradeCommType, SourceCommission, Account, OrderID, GTL, Trailer_Info, Clr
        in zip(Symbols2, Sides2, Cxls2, Qtys2, Prices2, Bids2, Asks2, T_DatS_DatTradeIDs2, TradeTiSSs2, Exbkrs2, 
                  TradeCommTypes2, SourceCommissions2, Accounts2, OrderIDs2, GTLs2, Trailer_Infos2, Clrs2)]
csv_info2

[('/17',
  '/17',
  '909224',
  '02',
  'CERS',
  'SS',
  'SS',
  'NEW',
  '100',
  '2.365000',
  '1706149900003',
  'STA=CERS',
  '0704',
  'PerUnit',
  'TERM',
  '0.010000',
  'AABBC'),
 ('CERS',
  'SS',
  'NEW',
  '100',
  '2.360000',
  '2.360000',
  '2.360000',
  '2.37000006/1406/191706149900003',
  '10:24:ContraSSFREX',
  '/17',
  '1706149900003',
  'STA=CERS',
  '0704',
  'PerUnit',
  'TERM',
  '0.010000',
  'AABBC'),
 ('CERS',
  'SS',
  'NEW',
  '100',
  '2.365000',
  '2.360000',
  '2.360000',
  '2.37000006/1406/191706149900003',
  '10:25:ContraSSFREX',
  '/17',
  '1706149900003',
  'STA=CERS',
  '0704',
  'PerUnit',
  'TERM',
  '0.010000',
  'AABBC'),
 ('CERS',
  'SS',
  'NEW',
  '100',
  '2.365000',
  '2.360000',
  '2.360000',
  '2.37000006/1406/191706149900003',
  '10:27:ContraSSFREX',
  '/17',
  '1706149900003',
  'STA=CERS',
  '0704',
  'PerUnit',
  'TERM',
  '0.010000',
  'AABBC'),
 ('CERS',
  'SS',
  'NEW',
  '100',
  '2.360000',
  '2.360000',
  '2.360000',
  '2.37000006/

In [294]:
csv_info = [(Symbol, Side, Cxl, Qty, Price, Bid, Ask, T_DatS_DatTradeID, TradeTiSS, Exbkr, 
                  TradeCommType, SourceCommission, Account, OrderID, GTL, Trailer_Info, Clr) 
        for Symbol, Side, Cxl, Qty, Price, Bid, Ask, T_DatS_DatTradeID, TradeTiSS, Exbkr, 
                  TradeCommType, SourceCommission, Account, OrderID, GTL, Trailer_Info, Clr
        in zip(Symbols, Sides, Cxls, Qtys, Prices, Bids, Asks, T_DatS_DatTradeIDs, TradeTiSSs, Exbkrs, 
                  TradeCommTypes, SourceCommissions, Accounts, OrderIDs, GTLs, Trailer_Infos, Clrs)]
csv_info


[('CERS',
  'SS',
  'NEW',
  '2.756',
  '2.400000',
  '2.340000',
  '2.340000',
  '2.45000006/1406/191706149900003',
  '09:30:CustSS',
  '/17',
  '/17',
  '00',
  '0704',
  'PerUnit',
  '0.010000',
  'AABBC',
  '1706149900003'),
 ('CERS',
  'SS',
  'NEW',
  '100',
  '2.36000018422.360000',
  '2.37000006/1406/191706149900003',
  '2.37000006/1406/191706149900003',
  '10:20:ContraSSFREX',
  '/17',
  '/17',
  '866087',
  '0704',
  'PerUnit',
  'TERM',
  'AABBC',
  '1706149900003',
  'STA=CERS'),
 ('CERS',
  'SS',
  'NEW',
  '200',
  '2.350000',
  '2.360000',
  '2.360000',
  '2.37000006/1406/191706149900003',
  '10:20:ContraSSFREX',
  '/17',
  '/17',
  '04',
  '0704',
  'PerUnit',
  '0.010000',
  'AABBC',
  '1706149900003'),
 ('CERS',
  'SS',
  'NEW',
  '100',
  '2.350000',
  '2.350000',
  '2.350000',
  '2.36000006/1406/191706149900003',
  '10:20:ContraSSFREX',
  '/17',
  '/17',
  '23',
  '0704',
  'PerUnit',
  '0.010000',
  'AABBC',
  '1706149900003'),
 ('CERS',
  'SS',
  'NEW',
  '100',
 

In [311]:
csv_info.append(csv_info2)
csv_info


[('CERS',
  'SS',
  'NEW',
  '2.756',
  '2.400000',
  '2.340000',
  '2.340000',
  '2.45000006/1406/191706149900003',
  '09:30:CustSS',
  '/17',
  '/17',
  '00',
  '0704',
  'PerUnit',
  '0.010000',
  'AABBC',
  '1706149900003'),
 ('CERS',
  'SS',
  'NEW',
  '100',
  '2.36000018422.360000',
  '2.37000006/1406/191706149900003',
  '2.37000006/1406/191706149900003',
  '10:20:ContraSSFREX',
  '/17',
  '/17',
  '866087',
  '0704',
  'PerUnit',
  'TERM',
  'AABBC',
  '1706149900003',
  'STA=CERS'),
 ('CERS',
  'SS',
  'NEW',
  '200',
  '2.350000',
  '2.360000',
  '2.360000',
  '2.37000006/1406/191706149900003',
  '10:20:ContraSSFREX',
  '/17',
  '/17',
  '04',
  '0704',
  'PerUnit',
  '0.010000',
  'AABBC',
  '1706149900003'),
 ('CERS',
  'SS',
  'NEW',
  '100',
  '2.350000',
  '2.350000',
  '2.350000',
  '2.36000006/1406/191706149900003',
  '10:20:ContraSSFREX',
  '/17',
  '/17',
  '23',
  '0704',
  'PerUnit',
  '0.010000',
  'AABBC',
  '1706149900003'),
 ('CERS',
  'SS',
  'NEW',
  '100',
 

In [317]:
csv_content = '/n'.join([",".join(l) for l in csv_info[:5]])

with open('sell_short_trades_final.csv','w') as outfile:
    outfile.write(csv_content)

!cat sell_short_trades_final.csv | head -n 5

CERS,SS,NEW,2.756,2.400000,2.340000,2.340000,2.45000006/1406/191706149900003,09:30:CustSS,/17,/17,00,0704,PerUnit,0.010000,AABBC,1706149900003/nCERS,SS,NEW,100,2.36000018422.360000,2.37000006/1406/191706149900003,2.37000006/1406/191706149900003,10:20:ContraSSFREX,/17,/17,866087,0704,PerUnit,TERM,AABBC,1706149900003,STA=CERS/nCERS,SS,NEW,200,2.350000,2.360000,2.360000,2.37000006/1406/191706149900003,10:20:ContraSSFREX,/17,/17,04,0704,PerUnit,0.010000,AABBC,1706149900003/nCERS,SS,NEW,100,2.350000,2.350000,2.350000,2.36000006/1406/191706149900003,10:20:ContraSSFREX,/17,/17,23,0704,PerUnit,0.010000,AABBC,1706149900003/nCERS,SS,NEW,100,2.350000,2.350000,2.350000,2.36000006/1406/191706149900003,10:22:ContraSSFREX,/17,/17,29,0704,PerUnit,0.010000,AABBC,1706149900003